In [2]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Arithmetic Asian Option

In [17]:
#Antithetic technique : to improve accuracy
def asian_out_option_arithmetic(S0, K, n, T, r, vol, N, M):
    """
    S0 : price of the underlying at time t=0
    K : strike price of the option
    n : number of observations
    T : time to maturity
    r : risk-free rate
    vol : volatility
    N : number of timesteps
    M : number of simulations
    """
    #Pricing Asian (arithmetic) out option
    dt=T/N
    period=N//n
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St1=S0
        St2=S0
        At1_sum=0
        At2_sum=0
        observation_count = 0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn1=St1*np.exp(nudt+volsdt*epsilon)
            Stn2=St2*np.exp(nudt-volsdt*epsilon)
            St1=Stn1
            St2=Stn2
            if (j + 1) % period == 0:
                At1_sum += St1
                At2_sum += St2
                observation_count += 1
            
        A1=At1_sum/n
        A2=At2_sum/n
        CT=(np.max(A1-K, 0)+np.max(A2-K, 0))/2
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [18]:
asian_out_option_arithmetic(100, 110, 12, 1, 0.02, 0.2, 100, 1000)

-8.817158254733533

### Geometric Asian Option

In [ ]:
def asian_out_geometric_option(S0, K, T1, T2, r, vol, N, M):
    #Pricing Asian (geometric) out option
    dt=(T2-T1)/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St1=S0
        St2=S0
        GT1=1
        GT2=1
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn1=St1*np.exp(nudt+volsdt*epsilon)
            Stn2=St2*np.exp(nudt-volsdt*epsilon)
            St1=Stn1
            St2=Stn2
            GT1=GT1*St1
            GT2=GT2*St2
            
        G1=GT1**(1/N)
        G2=GT2**(1/N)
        CT=(np.max(G1-K, 0)+np.max(G2-K))/2
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T2)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T2)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0